In [ ]:
import pyautogui
import pygetwindow as gw
import torch
from torchvision import transforms, models
from PIL import Image
import numpy as np
import cv2
import time

# === Load Model ===
MODEL_PATH = "emotion_model.pth"
checkpoint = torch.load(MODEL_PATH, map_location=torch.device('cpu'))
classes = checkpoint['label_encoder']

model = models.resnet18(pretrained=False)
model.fc = torch.nn.Linear(model.fc.in_features, len(classes))
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()

# === Image Transform ===
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5]*3, std=[0.5]*3)
])

def predict_emotion_from_pil(pil_img):
    image = transform(pil_img).unsqueeze(0)
    with torch.no_grad():
        output = model(image)
        probs = torch.nn.functional.softmax(output, dim=1).squeeze().numpy()
    return dict(zip(classes, probs))

# === Main Loop ===
def main():
    print("Looking for MS Paint window...")
    while True:
        try:
            paint = [w for w in gw.getWindowsWithTitle("Paint") if w.isActive][0]
            break
        except IndexError:
            print("Waiting for Paint to be active...")
            time.sleep(1)

    print(f"Tracking Paint window: {paint.title}")

    while True:
        # Take screenshot of Paint window
        if paint.isMinimized or not paint.isActive:
            time.sleep(0.5)
            continue
        bbox = (paint.left, paint.top, paint.width, paint.height)
        screenshot = pyautogui.screenshot(region=bbox)
        prediction = predict_emotion_from_pil(screenshot)

        # Convert to numpy image for OpenCV
        img_np = np.array(screenshot)
        img_bgr = cv2.cvtColor(img_np, cv2.COLOR_RGB2BGR)

        # Draw bars for predictions
        y_start = 30
        bar_width = 300
        for i, (emotion, score) in enumerate(sorted(prediction.items(), key=lambda x: -x[1])):
            bar_length = int(bar_width * score)
            y = y_start + i * 30
            cv2.rectangle(img_bgr, (10, y), (10 + bar_length, y + 20), (0, 255, 0), -1)
            cv2.putText(img_bgr, f"{emotion}: {score:.2f}", (10, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)

        cv2.imshow("Emotion Prediction (Paint)", img_bgr)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()
